In [1]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Dropout, LSTM, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

2023-03-12 14:09:18.809906: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
with open("intents.json") as file:
    data = json.load(file)

In [3]:
training_sentences = []   # holds the training data
training_labels = []      # holds the target labels correspond to the training data
labels = []               # holds the labels
responses = []            # holds the all the responses to the intents

for intent in data['intents']:
    for pattern in intent['patterns']:
        #print(pattern)
        training_sentences.append(pattern)
        #print(intent["tag"])
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    #print(intent["responses"])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
num_classes = len(labels)

In [4]:
# convert training_labels into model understandable form

label_enc = LabelEncoder()
#print(training_labels)
label_enc.fit(training_labels)
training_labels = label_enc.transform(training_labels)
#training_labels

In [5]:
# vectorize the text data corpus by using 'Tokenizer' class
# when we use this class - tby default it removes all punctuations marks
# turns the texts in space seperated sequences of words
# then split into lists of tokens. This wills indexed or vectorised

vocab_size = 1000      # maximum vocabulary size
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"    # out of token to deal with out of vocabulary words


tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen = max_len)

# pad_sequence method to make the text sequence into same size

In [6]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(Bidirectional(LSTM(64, activation='relu')))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(num_classes, activation='softmax'))

adam = tf.keras.optimizers.Adam(0.001)
model.compile(loss = 'sparse_categorical_crossentropy',
             optimizer = adam, metrics=['accuracy'])

model.summary()

2023-03-12 14:09:31.734762: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 bidirectional (Bidirectiona  (None, 128)              41472     
 l)                                                              
                                                                 
 dense (Dense)               (None, 32)                4128      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 31)                1023      
                                                                 
Total params: 62,623
Trainable params: 62,623
Non-trainable params: 0
____________________________________________________

In [7]:
epochs = 300
callback = tf.keras.callbacks.EarlyStopping(monitor = 'accuracy', patience = 10, start_from_epoch=200, restore_best_weights=True)
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs, callbacks=callback)

Epoch 1/300
5/5 [==============================] - 4s 22ms/step - loss: 3.4342 - accuracy: 0.0479
Epoch 2/300
5/5 [==============================] - 0s 21ms/step - loss: 3.4314 - accuracy: 0.0753
Epoch 3/300
5/5 [==============================] - 0s 23ms/step - loss: 3.4297 - accuracy: 0.0753
Epoch 4/300
5/5 [==============================] - 0s 25ms/step - loss: 3.4264 - accuracy: 0.0753
Epoch 5/300
5/5 [==============================] - 0s 24ms/step - loss: 3.4239 - accuracy: 0.0753
Epoch 6/300
5/5 [==============================] - 0s 23ms/step - loss: 3.4189 - accuracy: 0.0753
Epoch 7/300
5/5 [==============================] - 0s 23ms/step - loss: 3.4157 - accuracy: 0.0753
Epoch 8/300
5/5 [==============================] - 0s 22ms/step - loss: 3.4051 - accuracy: 0.0753
Epoch 9/300
5/5 [==============================] - 0s 24ms/step - loss: 3.3922 - accuracy: 0.0753
Epoch 10/300
5/5 [==============================] - 0s 23ms/step - loss: 3.3864 - accuracy: 0.0753
Epoch 11/300
5/5 [=

5/5 [==============================] - 0s 25ms/step - loss: 0.5920 - accuracy: 0.7877
Epoch 84/300
5/5 [==============================] - 0s 21ms/step - loss: 0.5549 - accuracy: 0.8219
Epoch 85/300
5/5 [==============================] - 0s 22ms/step - loss: 0.4923 - accuracy: 0.8425
Epoch 86/300
5/5 [==============================] - 0s 22ms/step - loss: 0.4641 - accuracy: 0.8562
Epoch 87/300
5/5 [==============================] - 0s 22ms/step - loss: 0.4903 - accuracy: 0.8151
Epoch 88/300
5/5 [==============================] - 0s 21ms/step - loss: 0.4319 - accuracy: 0.9110
Epoch 89/300
5/5 [==============================] - 0s 22ms/step - loss: 0.4828 - accuracy: 0.8493
Epoch 90/300
5/5 [==============================] - 0s 22ms/step - loss: 0.4297 - accuracy: 0.9041
Epoch 91/300
5/5 [==============================] - 0s 21ms/step - loss: 0.4458 - accuracy: 0.9041
Epoch 92/300
5/5 [==============================] - 0s 21ms/step - loss: 0.4199 - accuracy: 0.8904
Epoch 93/300
5/5 [=====

5/5 [==============================] - 0s 26ms/step - loss: 0.1470 - accuracy: 0.9452
Epoch 166/300
5/5 [==============================] - 0s 21ms/step - loss: 0.0939 - accuracy: 0.9658
Epoch 167/300
5/5 [==============================] - 0s 23ms/step - loss: 0.1446 - accuracy: 0.9521
Epoch 168/300
5/5 [==============================] - 0s 22ms/step - loss: 0.0988 - accuracy: 0.9863
Epoch 169/300
5/5 [==============================] - 0s 21ms/step - loss: 0.0756 - accuracy: 0.9863
Epoch 170/300
5/5 [==============================] - 0s 24ms/step - loss: 0.1022 - accuracy: 0.9658
Epoch 171/300
5/5 [==============================] - 0s 21ms/step - loss: 0.0813 - accuracy: 0.9795
Epoch 172/300
5/5 [==============================] - 0s 21ms/step - loss: 0.1059 - accuracy: 0.9589
Epoch 173/300
5/5 [==============================] - 0s 20ms/step - loss: 0.1060 - accuracy: 0.9589
Epoch 174/300
5/5 [==============================] - 0s 21ms/step - loss: 0.0536 - accuracy: 0.9863
Epoch 175/300


In [8]:
# saving model
model.save("my_model.h5", history)

import pickle

# Saving tokeniser
with open("tokenizer.pickle", 'wb') as token:
    pickle.dump(tokenizer, token, protocol=pickle.HIGHEST_PROTOCOL)
    
    
# saving fitted labelencoder
with open('label_encoder.pickle', 'wb') as enc:
    pickle.dump(label_enc, enc, protocol=pickle.HIGHEST_PROTOCOL)

In [9]:
pickle.dump(labels,open('labels.pkl','wb'))